In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from pandas import DatetimeIndex

def mergeData(df):
    features =pd.read_csv('features.csv')
    storesdata =pd.read_csv('stores.csv')
    df = pd.merge(df, features, on=['Store','Date','IsHoliday'],how='inner')
    df = pd.merge(df, storesdata, on=['Store'], how='inner')
    return df


In [2]:
merged_df = mergeData(pd.read_csv('train.csv'))

In [3]:
merged_df.fillna(value=0, inplace=True)

merged_df['Markdowns'] = merged_df['MarkDown1'] + merged_df['MarkDown2'] + merged_df['MarkDown3'] + merged_df['MarkDown4'] + merged_df['MarkDown5'] 
labelsToDrop = ['MarkDown1', 'MarkDown2', 'MarkDown3','MarkDown4','MarkDown5']
merged_df.drop(labels=labelsToDrop,axis=1, inplace=True)

In [4]:
merged_df.info()
merged_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 12 columns):
Store           421570 non-null int64
Dept            421570 non-null int64
Date            421570 non-null object
Weekly_Sales    421570 non-null float64
IsHoliday       421570 non-null bool
Temperature     421570 non-null float64
Fuel_Price      421570 non-null float64
CPI             421570 non-null float64
Unemployment    421570 non-null float64
Type            421570 non-null object
Size            421570 non-null int64
Markdowns       421570 non-null float64
dtypes: bool(1), float64(6), int64(3), object(2)
memory usage: 39.0+ MB


,Store,Dept,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Markdowns
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123,60.090059,3.361027,171.201947,7.960289,136727.915739,6684.041435
std,12.785297,30.492054,22711.183519,18.447931,0.458515,39.159276,1.863296,60980.583328,14750.941552
min,1.000000,1.000000,-4988.940000,-2.060000,2.472000,126.064000,3.879000,34875.000000,0.000000
25%,11.000000,18.000000,2079.650000,46.680000,2.933000,132.022667,6.891000,93638.000000,0.000000
50%,22.000000,37.000000,7612.030000,62.090000,3.452000,182.318780,7.866000,140167.000000,0.000000
75%,33.000000,74.000000,20205.852500,74.280000,3.738000,212.416993,8.572000,202505.000000,8075.260000
max,45.000000,99.000000,693099.360000,100.140000,4.468000,227.232807,14.313000,219622.000000,160510.610000


In [5]:
merged_df.loc[merged_df['Weekly_Sales']>300000,"Date"].value_counts()

df = merged_df
df.Date = pd.to_datetime(df.Date)

df['Year'] = DatetimeIndex(df['Date']).year
df['Month']= DatetimeIndex(df['Date']).month
df['Day'] = DatetimeIndex(df['Date']).day
df = df.drop(columns=['Date'])

In [6]:
#creating the feature matrix 
feature_cols = ['Store', 'Temperature','Fuel_Price','CPI','Unemployment', 'Markdowns', 'Size', 'Dept', 'Weekly_Sales', 'Year', 'Month', 'Day']
X = df[feature_cols]
y = df['IsHoliday']

In [7]:
decisiontree = DecisionTreeClassifier()

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [9]:
decisiontree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [10]:
y_predict_dt = decisiontree.predict(X_test)
print(y_predict_dt)

[ True False False ... False False False]


In [11]:
from sklearn.metrics import accuracy_score

score_dt = accuracy_score(y_test, y_predict_dt)
print(score_dt)

1.0
